<a href="https://colab.research.google.com/github/berfingundem/BladderCancer/blob/main/ViTCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_curve, auc
from sklearn.preprocessing import label_binarize
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import timm
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Google Drive bağlantısı
from google.colab import drive
drive.mount('/content/drive')

# Veri dizini (Google Drive)
data_dir = "/content/drive/MyDrive/bladder_set"
class_names = ['LGC', 'NTL', 'HGC', 'NST']  # Define your class names based on your folders

# Eğitim ve doğrulama klasörlerini oluşturma
train_dir = os.path.join(data_dir, "train")
val_dir = os.path.join(data_dir, "val")
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# Her sınıf için verileri ayırma (Eğitim ve doğrulama)
for class_name in class_names:
    class_path = os.path.join(data_dir, class_name)
    images = os.listdir(class_path)
    train_images, val_images = train_test_split(images, test_size=0.2, random_state=42)

    # Eğitim ve doğrulama klasörlerine taşıma
    train_class_dir = os.path.join(train_dir, class_name)
    val_class_dir = os.path.join(val_dir, class_name)
    os.makedirs(train_class_dir, exist_ok=True)
    os.makedirs(val_class_dir, exist_ok=True)

    for image in train_images:
        shutil.copy(os.path.join(class_path, image), os.path.join(train_class_dir, image))

    for image in val_images:
        shutil.copy(os.path.join(class_path, image), os.path.join(val_class_dir, image))

# Görsel dönüşümleri
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

# Eğitim ve doğrulama verileri
train_dataset = ImageFolder(os.path.join(data_dir, 'train'), transform=transform)
val_dataset = ImageFolder(os.path.join(data_dir, 'val'), transform=transform)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)
class_names = train_dataset.classes
num_classes = len(class_names)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=num_classes)
model.to(device)

# Eğitim bileşenleri
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

# Eğitim ayarları
num_epochs = 14
train_loss_list, val_loss_list = [], []
train_acc_list, val_acc_list = [], []

# Eğitim döngüsü
for epoch in range(num_epochs):
    model.train()
    train_loss, correct, total = 0, 0, 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)

    train_acc = correct / total
    train_loss /= len(train_loader)
    train_acc_list.append(train_acc)
    train_loss_list.append(train_loss)

    # Doğrulama
    model.eval()
    val_loss, correct, total = 0, 0, 0
    y_true, y_pred, y_score = [], [], []
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            probs = torch.softmax(outputs, dim=1)
            _, predicted = outputs.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)

            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())
            y_score.extend(probs.cpu().numpy())

    val_acc = correct / total
    val_loss /= len(val_loader)
    val_acc_list.append(val_acc)
    val_loss_list.append(val_loss)

    print(f"Epoch {epoch+1}/{num_epochs} | Train Acc: {train_acc:.2f} | Val Acc: {val_acc:.2f}")

# --- GÖRSEL 1: Loss ve Accuracy Eğrileri ---
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(train_acc_list, label='Train Accuracy')
plt.plot(val_acc_list, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Accuracy Eğrisi')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(train_loss_list, label='Train Loss')
plt.plot(val_loss_list, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss Eğrisi')
plt.legend()
plt.tight_layout()
plt.show()

# --- GÖRSEL 2: Konfüzyon Matrisleri (Ham & Normalize) ---
cm = confusion_matrix(y_true, y_pred)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

fig, axes = plt.subplots(1, 2, figsize=(14, 6), dpi=300)

# Ham değerler
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=class_names, yticklabels=class_names,
            cbar=False, annot_kws={"size": 12}, ax=axes[0])
axes[0].set_title("Confusion Matrix (Counts)", fontsize=14)
axes[0].set_xlabel("Predicted Label")
axes[0].set_ylabel("True Label")
axes[0].tick_params(axis='x', rotation=45)
axes[0].tick_params(axis='y', rotation=0)

# Normalize değerler
sns.heatmap(cm_normalized, annot=True, fmt=".2f", cmap="Blues",
            xticklabels=class_names, yticklabels=class_names,
            cbar=False, annot_kws={"size": 12}, ax=axes[1])
axes[1].set_title("Confusion Matrix (Normalized)", fontsize=14)
axes[1].set_xlabel("Predicted Label")
axes[1].set_ylabel("True Label")
axes[1].tick_params(axis='x', rotation=45)
axes[1].tick_params(axis='y', rotation=0)

plt.tight_layout()
plt.show()


# --- GÖRSEL 3: ROC Eğrisi ---
y_true_bin = label_binarize(y_true, classes=list(range(num_classes)))
y_score = np.array(y_score)
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(num_classes):
    fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

plt.figure(figsize=(8, 6), dpi=300)
for i in range(num_classes):
    plt.plot(fpr[i], tpr[i], label=f"{class_names[i]} (AUC = {roc_auc[i]:.2f})")
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Eğrisi')
plt.legend(loc='lower right')
plt.grid(True)
plt.tight_layout()
plt.show()
